In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



In [2]:
# Define device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transform for image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for ResNet
])

# Custom dataset class to load images manually
class CustomDataset(Dataset):
    def __init__(self, fire_dir, no_fire_dir, transform=None):
        self.fire_dir = fire_dir
        self.no_fire_dir = no_fire_dir
        self.transform = transform
        
        # Load image file paths
        self.fire_images = [os.path.join(fire_dir, f) for f in os.listdir(fire_dir) if f.lower().endswith(('jpg', 'jpeg', 'png'))]
        self.no_fire_images = [os.path.join(no_fire_dir, f) for f in os.listdir(no_fire_dir) if f.lower().endswith(('jpg', 'jpeg', 'png'))]

        # Labels: 0 for 'No Fire', 1 for 'Fire'
        self.images = self.fire_images + self.no_fire_images
        self.labels = [1] * len(self.fire_images) + [0] * len(self.no_fire_images)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        
        # Open the image
        img = Image.open(img_path).convert('RGB')
        
        # Apply the transform
        if self.transform:
            img = self.transform(img)
        
        return img, label



In [4]:
# Paths to the 'Fire' and 'No Fire' directories
fire_dir = 'FLAME3/FLAME 3 CV Dataset (Sycan Marsh)/Fire/RGB/Corrected FOV/'
no_fire_dir = 'FLAME3/FLAME 3 CV Dataset (Sycan Marsh)/No Fire/RGB/Corrected FOV/'

# Create dataset and dataloader
dataset = CustomDataset(fire_dir, no_fire_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

# Load the pre-trained ResNet model
model = models.resnet50(pretrained=False)  # Load ResNet50 without the pre-trained weights
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Replace the final fully connected layer to output 2 classes (fire, no fire)

# Load the trained model weights
model.load_state_dict(torch.load('flame_classification.pth'))
model.to(device)
model.eval()  # Set the model to evaluation mode



/local_scratch/slurm.1342496/ipykernel_767125/1900489302.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('flame_classification.pth'))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
# Initialize lists to store the true labels and predicted labels
y_true = []
y_pred = []

# Iterate over the data
with torch.no_grad():
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        # Collect true labels and predictions
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Print results
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


Accuracy: 84.28%
Precision: 0.84
Recall: 1.00
F1 Score: 0.91
